# RetinaNet for Signet Ring Cell Detection

In [0]:
#Check the GPU assigned on this GoogleColab runtime
!nvidia-smi

Wed May 27 10:57:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#We are using RetinaNet for our object detection, so we can clone the contents of this repository
!git clone https://github.com/fizyr/keras-retinanet.git

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 5736 (delta 12), reused 16 (delta 5), pack-reused 5705
Receiving objects: 100% (5736/5736), 13.37 MiB | 14.48 MiB/s, done.
Resolving deltas: 100% (3843/3843), done.


In [0]:
#cd to the keras-retinanet folder and install the library
%cd keras-retinanet/
!pip install .

/content/keras-retinanet
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=169025 sha256=5290911130892e5e603577a1620e34a8d9b033e521a47f9e24ce25a25fcf2c63
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13346 sha256=32eb9e77c8529ac786d0172a14d66b0af4c0af4213351f73d98cd19686c0becd
  Stored in directory: /root/.cache/pip/wheels/80/dd/ac/842235b63dddac12faa4b48ebe58b8944e8c2e57c2e38dddb6
Successfully built keras-retinanet keras-resnet


In [0]:
#Now install the C extensions which are required for RetinaNet
!python setup.py build_ext --inplace

running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/keras-retinanet/keras_retinanet/utils/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'keras_retinanet.utils.compute_overlap' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/keras_retinanet
creating build/temp.linux-x86_64-3.6/keras_retinanet/utils
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -c keras_retinanet/utils/compute_overlap.c -o build/temp.linux-x86_64-3.6/keras_retinanet/utils/compute_over

In [0]:
#Install tensorflow-gpu in order to use TensorFlow 2
!pip install tensorflow-gpu

     |████████████████████████████████| 516.2MB 32kB/s 


In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 1

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# Mount Contents of My Drive
from google.colab import drive
drive.mount("/content/drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
#Make a folder called "snapshots"
os.makedirs("snapshots", exist_ok=True)

In [0]:
#Make a folder called "WSI_Images"
os.makedirs("WSI_Images", exist_ok=True)

## Preprocessing Dataset for RetinaNet

The expected format of each line is:

path/to/image.jpg || x1 || y1 || x2 || y2 || class_name

In [0]:
#Read CSV file dataset annotations to WSI dataframe
WSI_df = pd.read_csv('/content/drive/My Drive/QMUL/Project/XML_to_CSV/WSI_Annotations.csv')

# WSI_df = pd.read_csv('/content/WSI_Annotations.csv')

In [0]:
#Define a function to show the image in the created dataframe
def show_image_objects(image_row):

  img_path = image_row.image_name
  box = [image_row.x_min, image_row.y_min, image_row.x_max, image_row.y_max]

  image = read_image_bgr('/content/drive/My Drive/QMUL/Project/Signet_ring_cell_dataset-20190621T091454Z-001/sig-train-pos/{}'.format(img_path))

  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  draw_box(draw, box, color=(255, 255, 0))

  plt.axis('off')
  plt.imshow(draw)
  plt.show()

In [0]:
#Show first 3 images in dataset
for i in range(3):
  show_image_objects(WSI_df.iloc[i])

Output hidden; open in https://colab.research.google.com to view.

In [0]:
#Split dataset into train and test sets
train_df, test_df = train_test_split(WSI_df, test_size=0.2, random_state=RANDOM_SEED)

In [0]:
train_df['image_name'] = 'WSI_Images/' + train_df['image_name'].astype(str)
test_df['image_name'] = 'WSI_Images/' + test_df['image_name'].astype(str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
#Check correct shape of train and test dataset splits
print('train shape:', train_df.shape)
print('test shape:', test_df.shape)

train shape: (7768, 6)
test shape: (1942, 6)


In [0]:
#See what train dataframe looks like
train_df.head()

,image_name,x_min,y_min,x_max,y_max,class_name
2772,WSI_Images/2018_67251_1-13_2019-02-26 00_53_32...,1101,1143,1160,1197,ring_cell_cancer
215,WSI_Images/2018_64982_1-3_2019-02-25 21_57_36-...,1008,24,1084,108,ring_cell_cancer
8102,WSI_Images/D20180627801_2019-05-21 18_40_34-lv...,829,274,905,348,ring_cell_cancer
4866,WSI_Images/2018_68805_1-7_2019-03-14 23_55_06-...,921,882,969,948,ring_cell_cancer
4106,WSI_Images/2018_68000_1-8_2019-02-26 02_28_01-...,779,211,843,272,ring_cell_cancer


In [0]:
#See what test dataframe looks like
test_df.head()

,image_name,x_min,y_min,x_max,y_max,class_name
1924,WSI_Images/2018_67251_1-12_2019-02-26 00_47_47...,687,1770,741,1802,ring_cell_cancer
6479,WSI_Images/2018_69188_1-1_2019-03-14 23_40_58-...,778,917,840,987,ring_cell_cancer
956,WSI_Images/2018_67251_1-3_2019-02-26 00_02_22-...,181,650,289,771,ring_cell_cancer
229,WSI_Images/2018_64982_1-3_2019-02-25 21_57_36-...,1377,1560,1454,1642,ring_cell_cancer
7065,WSI_Images/2018_69188_1-2_2019-03-14 23_35_35-...,960,862,1020,917,ring_cell_cancer


In [0]:
#Define the two files we will write into
ANNOTATIONS_FILE = 'annotations.csv'
CLASSES_FILE = 'classes.csv'

In [0]:
#Write the training data into the annotations file and do not include header and index as we are converting a dataframe
train_df.to_csv(ANNOTATIONS_FILE, index=False, header=None)

In [0]:
#Specify the classes that we have
classes = set(['ring_cell_cancer'])

with open(CLASSES_FILE, 'w') as f:
  for i, line in enumerate(sorted(classes)):
    f.write('{}, {}\n'.format(line, i))

In [0]:
#Check the correct format of created csv file
!head annotations.csv

WSI_Images/2018_67251_1-13_2019-02-26 00_53_32-lv0-15981-26268-2023-2014.jpeg,1101,1143,1160,1197,ring_cell_cancer
WSI_Images/2018_64982_1-3_2019-02-25 21_57_36-lv0-38368-62991-2040-2016.jpeg,1008,24,1084,108,ring_cell_cancer
WSI_Images/D20180627801_2019-05-21 18_40_34-lv0-12112-37198-2058-2042.jpeg,829,274,905,348,ring_cell_cancer
WSI_Images/2018_68805_1-7_2019-03-14 23_55_06-lv0-42909-22296-2027-2043.jpeg,921,882,969,948,ring_cell_cancer
WSI_Images/2018_68000_1-8_2019-02-26 02_28_01-lv0-83128-38504-2005-2016.jpeg,779,211,843,272,ring_cell_cancer
WSI_Images/2018_64982_1-3_2019-02-25 21_57_36-lv0-39555-58583-2006-2007.jpeg,1823,1271,1948,1372,ring_cell_cancer
WSI_Images/2018_68000_1-8_2019-02-26 02_28_01-lv0-83128-38504-2005-2016.jpeg,319,1737,359,1777,ring_cell_cancer
WSI_Images/2018_68000_1-8_2019-02-26 02_28_01-lv0-85836-40477-2011-2022.jpeg,859,1388,942,1464,ring_cell_cancer
WSI_Images/D20180628401_2019-05-21 18_36_19-lv0-16939-29532-2036-2080.jpeg,641,64,694,113,ring_cell_cancer
W

## Training the Model with the Dataset

In [0]:
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

print('Downloaded pre-trained model to ' + PRETRAINED_MODEL)

Downloaded pre-trained model to ./snapshots/_pretrained_model.h5


In [0]:
!keras_retinanet/bin/train.py --freeze-backbone --random-transform --weights {PRETRAINED_MODEL} \
  --batch-size 8 --steps 500 --epochs 10 csv annotations.csv classes.csv

Using TensorFlow backend.
Creating model, this may take a second...
2020-05-27 11:07:00.564213: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-27 11:07:00.607032: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-27 11:07:00.607644: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-05-27 11:07:00.607965: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-27 11:07:00.609841: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-05

In [0]:
!ls snapshots/

_pretrained_model.h5  resnet50_csv_04.h5  resnet50_csv_08.h5
resnet50_csv_01.h5    resnet50_csv_05.h5  resnet50_csv_09.h5
resnet50_csv_02.h5    resnet50_csv_06.h5  resnet50_csv_10.h5
resnet50_csv_03.h5    resnet50_csv_07.h5


In [0]:
model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])

In [0]:
model_path

'snapshots/resnet50_csv_10.h5'

In [0]:
model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

Using TensorFlow backend.


tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 , -14.25438 ,  28.50876 ,  14.25438 ],
       [-35.918785, -17.959393,  35.918785,  17.959393],
       [-16.      , -16.      ,  16.      ,  16.      ],
       [-20.158737, -20.158737,  20.158737,  20.158737],
       [-25.398417, -25.398417,  25.398417,  25.398417],
       [-11.313708, -22.627417,  11.313708,  22.627417],
       [-14.25438 , -28.50876 ,  14.25438 ,  28.50876 ],
       [-17.959393, -35.918785,  17.959393,  35.918785]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-45.254833, -22.627417,  45.254833,  22.627417],
       [-57.01752 , -28.50876 ,  57.01752 ,  28.50876 ],
       [-71.83757 , -35.918785,  71.83757 ,  35.918785],
       [-32.      , -32.      ,  32.      ,  32.      ],
       [-40.317474, -40.317474,  40.317474,  40.317474],
       [-50.796833, -50.796833,  50.7

In [0]:
labels_to_names = pd.read_csv(CLASSES_FILE, header=None).T.loc[0].to_dict()

In [0]:
labels_to_names

{0: 'ring_cell_cancer'}

In [0]:
THRES_SCORE = 0.3

def show_detected_objects(image_row):
  img_path = image_row.image_name
  true_box = [image_row.x_min, image_row.y_min, image_row.x_max, image_row.y_max]
  image = read_image_bgr(img_path)

  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  image = preprocess_image(image)
  image, scale = resize_image(image)

  boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))

  boxes /= scale

  draw_box(draw, true_box, color=(255, 255, 0))

  for box, score, label in zip(boxes[0], scores[0], labels[0]):
    if score < THRES_SCORE:
      break
    
    color = label_color(label)

    b = box.astype(int)
    draw_box(draw, b, color=color)

    caption = "{} {:.3f}".format(labels_to_names[label], score)
    draw_caption(draw, b, caption)

  plt.axis('off')
  plt.imshow(draw)
  plt.show()

In [0]:
test_df

,image_name,x_min,y_min,x_max,y_max,class_name
1924,WSI_Images/2018_67251_1-12_2019-02-26 00_47_47...,687,1770,741,1802,ring_cell_cancer
6479,WSI_Images/2018_69188_1-1_2019-03-14 23_40_58-...,778,917,840,987,ring_cell_cancer
956,WSI_Images/2018_67251_1-3_2019-02-26 00_02_22-...,181,650,289,771,ring_cell_cancer
229,WSI_Images/2018_64982_1-3_2019-02-25 21_57_36-...,1377,1560,1454,1642,ring_cell_cancer
7065,WSI_Images/2018_69188_1-2_2019-03-14 23_35_35-...,960,862,1020,917,ring_cell_cancer
...,...,...,...,...,...,...
3424,WSI_Images/2018_68000_1-7_2019-02-26 02_22_15-...,359,1787,406,1834,ring_cell_cancer
8760,WSI_Images/G1900703-2_2019-04-30 09_53_59-lv0-...,588,907,635,949,ring_cell_cancer
2768,WSI_Images/2018_67251_1-13_2019-02-26 00_53_32...,1141,1673,1220,1733,ring_cell_cancer
9568,WSI_Images/G1901976_2019-04-30 14_03_43-lv0-42...,1564,89,1630,181,ring_cell_cancer


In [0]:
#Predictions
for i in range(10):
  show_detected_objects(test_df.iloc[i])